This algorithm ,it shows top 5 similar restaurant when you give location and restaurant .
I want to use New Delphi beacuse it has more restaurant than other cities

I used some NLP feautes for text mining of recommendation systems
1.  Feature Extraction 
2. TF-IDF (with Aggregate rating.Used to incerasing success) 
3. Cosine Similarity


Aggregate rating added for your selection tools because maybe diffrent restaurant is close similarty score.But  smaller similarty score of restaurant is bigger aggregate rating than others.  




In [ ]:
import numpy as np 
import pandas as pd 
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk.tokenize import word_tokenize


data=pd.read_csv("../input/zomato.csv",encoding="iso-8859-9")  

In [ ]:
data.head(2)

In [ ]:
data['City'].value_counts(dropna = False).head(5)   

In [ ]:
data_city =data.loc[data['City'] == 'New Delhi']
data_new_delphi=data_city[['Restaurant Name','Cuisines','Locality','Aggregate rating']]

In [ ]:
data_new_delphi['Locality'].value_counts(dropna = False).head(5)   

In [ ]:
data_new_delphi.loc[data['Locality'] == 'Connaught Place']
data_new_delphi['Locality'].value_counts(dropna = False).head(5)   


In [ ]:
data_sample=[]

In [ ]:
def data_show(location,title):   
    
    # these variable has to global because of i want use some properties out of function for analysis  
    global data_sample       
    global cosine_sim
    global sim_scores
    global tfidf_matrix
    global corpus_index
    global feature
    global rest_indices
    global idx
    
    # When location comes from function ,our new data consist only location dataset
    data_sample = data_new_delphi.loc[data_new_delphi['Locality'] == location]  
    
    # index will be reset for cosine similarty index because Cosine similarty index has to be same value with result of tf-idf vectorize
    data_sample.reset_index(level=0, inplace=True) 
      
    #Feature Extraction
    data_sample['Split']="X"
    for i in range(0,data_sample.index[-1]):
        split_data=re.split(r'[,]', data_sample['Cuisines'][i])
        for k,l in enumerate(split_data):
            split_data[k]=str.lower(split_data[k].replace(" ", ""))
        split_data=' '.join(split_data[:])
        data_sample['Split'].iloc[i]=split_data
        
    ## --- TF - IDF Vectorizer---  ##
    #Extracting Stopword
    tfidf = TfidfVectorizer(stop_words='english')

    #Replace NaN for empty string
    data_sample['Split'] = data_sample['Split'].fillna('')

    # Applying TF-IDF Vectorizer
    tfidf_matrix = tfidf.fit_transform(data_sample['Split'])
    
    # Using for see Cosine Similarty scores
    feature= tfidf.get_feature_names()

    
    ## ---Cosine Similarity--- ##
    # Compute the cosine similarity matrix
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix) 
    
    # Column names are using for index
    corpus_index=[n for n in data_sample['Split']]
       
    #Construct a reverse map of indices    
    indices = pd.Series(data_sample.index, index=data_sample['Restaurant Name']).drop_duplicates() 
    
    #index of the restaurant matchs the cuisines
    idx = indices[title]

    
    #Aggregate rating added with cosine score in sim_score list.
    sim_scores=[]
    for i,j in enumerate(cosine_sim[idx]):
        k=data_sample['Aggregate rating'].iloc[i]
        if j != 0 :
            sim_scores.append((i,j,k))
            
    #Sort the restaurant names based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: (x[1],x[2]) , reverse=True) 

    # 5 similarty cuisines
    sim_scores = sim_scores[0:6] 

    rest_indices = [i[0] for i in sim_scores] 
  
    data_x =data_sample[['Restaurant Name','Aggregate rating']].iloc[rest_indices]
    
    data_x['Cosine Similarity']=0
    for i,j in enumerate(sim_scores):
        data_x['Cosine Similarity'].iloc[i]=round(sim_scores[i][1],2)
   
    return data_x
    

In [ ]:
# Top 5 similar restaurant with cuisine of 'Barbeque Nation' restaurant in Connaught Place
data_show('Connaught Place','Barbeque Nation')  ## location & Restaurant Name

*Delhi Darbar Dhaba* similarity score is bigger than Fa Yian but it is aggreagate rating is smaller than DD Dahaba. So you want to go Fa Yian rest.

In [ ]:
# it shows features of tf-idf matrix
data_tfidf=pd.DataFrame(tfidf_matrix.todense(),index=corpus_index, columns=feature)
data_tfidf.head(10)

In [ ]:
cosine_sim[96]  ## cosine similarity score .(Barbeque Nation = 96) 

In [ ]:
# Top 5 similar restaurant with cuisine of 'Biryani Blues' restaurant in Connaught Place
data_show('Connaught Place','Healthy Routes')   ## location & Restaurant Name

In [ ]:
data_tfidf=pd.DataFrame(tfidf_matrix.todense(),index=corpus_index, columns=feature)
data_tfidf.head(10)

In [ ]:
cosine_sim[104]  ## cosine similarity score (Healthy Routes = 104) 